In [1]:
# import requests
# import json
# import base64
# import ast

# def wav_to_base64string(file_path):
#     return base64.b64encode(open(file_path, "rb").read()).decode('utf-8')

# url = "https://ai4b-dev-asr.ulcacontrib.org/asr/v1/recognize/te"

# def get_payload(base64_audio_content):
#     payload = json.dumps({
#         "config": {
#             "language": {
#                 "sourceLanguage": "te"},
#                 "transcriptionFormat": {
#                     "value": "transcript"},
#                     "audioFormat": "wav"
#                     },
#                     "audio": [{"audioContent": f"{base64_audio_content}"}]})
#     return payload

# headers = {'Content-Type': 'application/json'}

In [2]:
import time
from tqdm import tqdm
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="bnriiitb/whisper-small-te-146h",language='Telugu')

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import numpy as np
import pandas as pd
import os
from IPython import display
import datasets
from datasets.info import DatasetInfosDict
from datasets import load_dataset,concatenate_datasets, DatasetDict, Dataset, Audio

/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import datasets
from datasets import load_dataset, DatasetDict, Dataset, Audio

ds = load_dataset("../data/exp_data/")

Resolving data files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 325165.31it/s]
Using custom data configuration exp_data-ebc60f7d09fea331
Found cached dataset audiofolder (/raid/cs20mds14030/.cache/huggingface/datasets/audiofolder/exp_data-ebc60f7d09fea331/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 463.56it/s]


In [5]:
ds = ds['train']
ds

Dataset({
    features: ['audio'],
    num_rows: 99
})

In [6]:
display.Audio('/raid/cs20mds14030/telugu_asr/data/exp_data/broken-vad_01_E8KRQoNpp8Y-0.wav')

In [7]:
# for f in ds:
#     print(f['audio']['path'])

In [8]:
import torch
from transformers import AutoProcessor, AutoModelForCTC
from tqdm import tqdm

processor = AutoProcessor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-telugu-tem-100")
model = AutoModelForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-telugu-tem-100")

In [9]:
sampling_rate = ds.features["audio"].sampling_rate

def transcribe(audio_array):
    inputs = processor(audio_array, sampling_rate=sampling_rate, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe speech
    transcription = processor.batch_decode(predicted_ids)
    transcription = transcription[0].replace("<s>","")
    return transcription



transcriptions_vk = []
transcriptions_wp = []
for item in tqdm(ds['audio']):
    transcriptions_vk.append(transcribe(item['array']))
    transcriptions_wp.append(pipe(item['array'],language='Telugu')['text'])

# print(transcriptions)

  0%|                                                                                                                                   | 0/99 [00:00<?, ?it/s]It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
/raid/cs20mds14030/miniconda3/envs/telugu_asr/lib/python3.8/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [24:33<00:00, 14.88s/it]


In [10]:
file_ids = [audio['path'].split('exp_data')[1][1:].replace(".wav","") for audio in ds['audio']]
file_ids[:2]

['broken-vad_00_lshQKEpvm6c-7', 'broken-vad_01_E8KRQoNpp8Y-0']

In [11]:
import pandas as pd
output_df = pd.DataFrame()

output_df.loc[:,'file_name'] = file_ids
output_df.loc[:,'vakyansh_transcription'] = transcriptions_vk
output_df.loc[:,'fine_tuned_whisper_transcription'] = transcriptions_wp
output_df.to_csv('exp_data_vakyansh.csv',index=False)
output_df

,file_name,vakyansh_transcription,fine_tuned_whisper_transcription
0,broken-vad_00_lshQKEpvm6c-7,మీరర,వాడు మాయము దిట్టిన్ సార్ వాడు మాయము దిట్టిన్ స...
1,broken-vad_01_E8KRQoNpp8Y-0,హలో యూవస్ వెల్కంగ్ర తెలిగవయూట్యూబ్ ఛానల్ కంపెన...,హెల్వ్ వ్యూవర్స్ వెల్కంస్తుగ్రాలు తెలిగివి యూట...
2,broken-vad_01__5BntK9iyo4-2,తమ జీవితాల్లోకి కొత్త ఆనందాన్ని తీసుకుని వచ్చి...,తమ జీవితంలోకి కొత్త ఆనందాన్ని తీసుకొని వచ్చిన ...
3,broken-vad_02_WSrcQ5d4eGs-0,ఏఆగాగ క్కడోట రే,ఆగా కల్పోతున్న వేటి లోపులు కాలడానికి ఇల్లేదు ఇ...
4,broken-vad_02_YvnbpgyNEG0-2,అంటే ఆల్మోస్ట్ అన్ని జిల్లాల్లోనూ మన విశాఖపట్న...,అల్మోస్టు అన్ని జిల్లాల్లోనూ విశాఖపట్నం నుంచి ...
...,...,...,...
94,vad_57_0OR5H_QiQD8,ఎందుకంటే అలిల్ మీ బొబ్బర గుడాల్ సింగపూల సంద సద...,చెందుగారు అంటే అల్చాల్మే బొబ్బరగుడాలు సింగపుల ...
95,vad_66_Yk7PuNPW7F4,అబియస్గా హిలహోల్ కంట్రోల్ హిల్ డీసెంట్ కంట్రోల...,హిల్ హుల్ కంట్రోల్ కంట్రోల్ కొన్ని ఎక్స్టా ఫీ...
96,vad_69_0OR5H_QiQD8,అరే మా బా సింగపూర్ ససరన ఏ మాట్లాడుతున్నారా,మా బాస్ సింగపూలో సందించి నుండి ఏమార్లాడుతున్నారా
97,vad_71_ChbBFnO-XRU,ానీ,ప్రాంక్ చేయని కానీ


In [12]:
exp_data = pd.read_csv("../data/New_Exp_Data.csv")
exp_data.head()

,file_name,length (in sec),speaker_gender,no_of_speakers,code_mixed,code_mixed_languages,dialect,background noise/music,theme
0,vad_00_OY9xCzgKIJg,17.55,female,1,no,na,telangana,no,weather
1,broken-vad_02_YvnbpgyNEG0-5,14.33,male,1,no,na,nuetral,no,weather
2,broken-vad_02_YvnbpgyNEG0-4,20.00,male,1,no,na,nuetral,no,weather
3,broken-vad_02_YvnbpgyNEG0-2,20.00,female,1,yes,telugu+english,costal andhra,no,weather
4,vad_00_JHFcHueFxBw,8.22,female,1,yes,telugu+english,neutral,yes,weather


In [13]:
final_df = exp_data.merge(output_df)

In [14]:
final_df.to_csv('final_df.csv',index=False)

In [42]:
import evaluate
metric = evaluate.load("wer")

In [43]:
import numpy as np
import pandas as pd

In [44]:
labelled_df = pd.read_csv('../data/labelled_data_57.csv')
cols_to_use = [col for col in labelled_df.columns if col.find("Unnamed")==-1]
labelled_df = labelled_df[cols_to_use]
labelled_df.head()

,file_name,length (in sec),speaker_gender,no_of_speakers,code_mixed,code_mixed_languages,dialect,background noise/music,theme,vakyansh_transcription,fine_tuned_whisper_transcription,ground_truth,Google Cloud Speech to Text,Azure Speech to Text
0,vad_12_R4kPm3KHUcw,8.97,male,1.0,no,telugu,kadapa,no,general,రేమచ్య ఏరా అంటే ఏమైందిరా అంటే ఎమ్మెల్య కొడుకు నేవి పేస్తనిూ్తంటే భయ ఫోన్ చేస్తాండా,మచే యాండోర్ అంటే ఏమైంది అంటే ఏమినుది అమ్మీలు కుడుకు నేను వికరాబాదడిలో చంపేస్తుండానే ఏమో కూతులు గూస్తాం అంటే బైపుడు ఫోన్ చేస్తాండా,రేయ్ మచా యాడుండవ్ రా అంటే ఏమైంది రా అంటే ఎం లెదురా నిన్ను ఎం యల్ ఎ కొడుకు నిన్ను వికారాబాద్ అడవిలో చంపేస్తుండా అని ఏవో కూతులు కూస్తా ఉంటే బయపడి పోన్ చెస్తాఉండా,రేయ్ మచ్చా ఏడున్నావ్ రా అంటే ఏమైంది రా అంటే ఎవరు ఎమ్మెల్యే కొడుకు నేను వికారాబాద్ చంపేస్తుంటే చూస్తూ ఉంటే ఫోన్ చేస్తా ఉండు,NaN
1,vad_35_R4kPm3KHUcw,3.03,male,1.0,no,telugu,kadapa,no,general,ఇక్కడ మనలాగా అక్కడపతథియేటర్లో ఎక్కడ ఉంటాయి,ఇక్కడ మునలాగా అక్కడి భర్త కట్ థియేటర్లో ఎక్కడో ఉంటాయి,ఇక్కడ మనలాగ ఎక్కడపడ్తే అక్కడ థియేటర్లు ఉండవు ఎక్కడో ఉంటాయి,ఇక్కడ మన లాగా ఇక్కడికొచ్చాక థియేటర్లో ఎక్కడ ఉంటాయి,NaN
2,broken-vad_05_lshQKEpvm6c-8,13.22,male,2.0,yes,telugu+english,kadapa,yes,general,నుేరా,అమ్మ ఇంతసాయకువగారు,నీయమ్మ ఎంత సైకోగడురా వీడు నువ్వేరా కాదురా ఇటురా దా ముందుకురా ఇదేందిరా రాసినవ్ ఫస్ట్ పేపరులో,ఈ అబ్బాయి ఎంత సేపు పడుతుంది ఫస్ట్ పేపర్ లో,NaN
3,broken-vad_00_lshQKEpvm6c-7,19.81,male,2.0,yes,telugu+english,mahabub nagar,yes,general,మీరర,వాడు మాయము దిట్టిన్ సార్ వాడు మాయము దిట్టిన్ సార్ వాడు మాయము దిట్టిన్ సార్ వాడు దిట్టిన్ సార్,వాడే చేసిండు సార్ ఎమైందిరా సార్ ఈడు మా అమ్మను తిట్టిండు సార్ వాడు మా అమ్మను తిట్టిండు సార్ లేదు సార్ ఈడే మా అమ్మను తిట్టిండు సార్ లేదు సార్ ఈడే మా అమ్మను తిట్టిండు సార్ ఫస్ట్ ఎవ్వరా తిట్టింది ఈడు తిట్టిండు సార్ వాడు తిట్టిండు సార్ ఇంతకే మీరెవ్వరా అన్నదమ్ములం సార్ బ్యాగ్రౌంద్ అయిపోయింది,ఏమైంది అయిపోయింది,NaN
4,broken-vad_05_lshQKEpvm6c-0,20.00,male,1.0,yes,telugu+english,mahabub nagar,yes,general,మొత్తం న్ ెట్టప్పా స్కూల్,మొత్తం నల్బై తెల్లా పీపుర్లు వీటినారు మీరు గాస్తుంది అని వెట్టారు పీపుర్ నీటింటారు మార్గులిస్తున్నారు నల్బై పీపుర్ నీటింటి అంటే రెండరుండటి ఎన్నిమార్కులు పాస్,ఎం రాసినవ్రా మొత్తం నల్బై తెల్లా పేపర్లు పెట్టినవ్ మీరే కద సార్ నిన్న చెప్పారు పేపర్ నీటుగా ఉంటె రొండు మార్కులు ఇస్తమన్నరు నల్బై పేపర్లు నీటుగున్నయ్ అంటే రొండు రొండు అంటే యెనబై మార్కులు పాస్ ఏంటీ ఇచ్చిపడెయ్యలి సార్ మనము తెలుసు కదా హైస్కూల్ బెత్తం తేసాడురోయ్,ఏం రా మొత్తం 40 వేల భలే పెట్టావ్ అని చెప్పారు మరి ఇస్తున్నారు నీకు ఇచ్చాడు సర్ మనకు తెలిసిన హై స్కూల్,NaN


In [45]:
labelled_df.dropna(subset='dialect',inplace=True)

In [46]:
labelled_df.dialect.value_counts(normalize=True)

neutral          0.375000
telangana        0.196429
srikakulam       0.107143
nelluru          0.071429
kadapa           0.053571
muslim telugu    0.053571
chitoor          0.053571
mahabub nagar    0.035714
nuetral          0.035714
costal andhra    0.017857
Name: dialect, dtype: float64

In [47]:
dialect = 'neutral'
def benchmark_wer(dialect):
    gt = labelled_df[labelled_df.dialect==dialect]['ground_truth'].values.tolist()
    vk = labelled_df[labelled_df.dialect==dialect]['vakyansh_transcription'].values.tolist()
    gcp = labelled_df[labelled_df.dialect==dialect]['Google Cloud Speech to Text'].values.tolist()
    fw = labelled_df[labelled_df.dialect==dialect]['fine_tuned_whisper_transcription'].values.tolist()
    
    vk_wer = gcp_wer = fw_wer = 100
    try:
        vk_wer = metric.compute(predictions=vk, references=gt)
        gcp_wer = metric.compute(predictions=gcp, references=gt)
        fw_wer = metric.compute(predictions=fw, references=gt)
    except Exception:
        print(f'Erorr occured while computing WER for {dialect}')
    return vk_wer, gcp_wer, fw_wer 

In [88]:
labelled_df.dialect = labelled_df.dialect.replace('nuetral','neutral')

In [89]:
wer_results = []
for dialect in labelled_df.dialect.unique():
    vk,gcp,fw = benchmark_wer(dialect)
    wer_results.append([dialect,vk,fw,gcp])
    # print(f'Dialect --> {dialect}, Vakyansh --> {vk}, Fine Tuned Whisper --> {fw}, GCP --> {gcp}')
wer_results    

[['kadapa', 0.8958333333333334, 0.8541666666666666, 0.7708333333333334],
 ['mahabub nagar', 0.9876543209876543, 0.8518518518518519, 0.9629629629629629],
 ['muslim telugu', 0.8363636363636363, 0.6, 0.8545454545454545],
 ['nelluru', 0.9652173913043478, 0.7652173913043478, 0.8695652173913043],
 ['neutral', 0.5720653789004457, 0.4947994056463596, 0.5586924219910847],
 ['chitoor', 0.8518518518518519, 0.6666666666666666, 0.8518518518518519],
 ['costal andhra',
  0.6756756756756757,
  0.40540540540540543,
  0.7027027027027027],
 ['srikakulam', 0.9261744966442953, 0.8859060402684564, 0.959731543624161],
 ['telangana', 0.8278145695364238, 0.7549668874172185, 0.8278145695364238]]

In [91]:
wer_df = pd.DataFrame(wer_results,columns=['Dialect','Vakyansh WER', 'Finetuned Wishper WER','GCP WER'])

for col in wer_df.select_dtypes('number').columns:
    wer_df[col] = wer_df[col].apply(lambda x:round(100*x,2))

dialect_dist_df = labelled_df.groupby('dialect')['file_name'].count().reset_index(name='# Samples')
wer_df = wer_df.merge(dialect_dist_df,left_on='Dialect',right_on='dialect')
wer_df.drop('dialect',axis=1,inplace=True)
wer_df = wer_df[['Dialect','# Samples' ,'Vakyansh WER', 'Finetuned Wishper WER', 'GCP WER']]
wer_df

,Dialect,# Samples,Vakyansh WER,Finetuned Wishper WER,GCP WER
0,kadapa,3,89.58,85.42,77.08
1,mahabub nagar,2,98.77,85.19,96.30
2,muslim telugu,3,83.64,60.00,85.45
3,nelluru,4,96.52,76.52,86.96
4,neutral,23,57.21,49.48,55.87
5,chitoor,3,85.19,66.67,85.19
6,costal andhra,1,67.57,40.54,70.27
7,srikakulam,6,92.62,88.59,95.97
8,telangana,11,82.78,75.50,82.78


In [49]:
pd.set_option('display.max_colwidth', None)

In [61]:
labelled_df[labelled_df.dialect=='kadapa']

,file_name,length (in sec),speaker_gender,no_of_speakers,code_mixed,code_mixed_languages,dialect,background noise/music,theme,vakyansh_transcription,fine_tuned_whisper_transcription,ground_truth,Google Cloud Speech to Text,Azure Speech to Text
0,vad_12_R4kPm3KHUcw,8.97,male,1.0,no,telugu,kadapa,no,general,రేమచ్య ఏరా అంటే ఏమైందిరా అంటే ఎమ్మెల్య కొడుకు నేవి పేస్తనిూ్తంటే భయ ఫోన్ చేస్తాండా,మచే యాండోర్ అంటే ఏమైంది అంటే ఏమినుది అమ్మీలు కుడుకు నేను వికరాబాదడిలో చంపేస్తుండానే ఏమో కూతులు గూస్తాం అంటే బైపుడు ఫోన్ చేస్తాండా,రేయ్ మచా యాడుండవ్ రా అంటే ఏమైంది రా అంటే ఎం లెదురా నిన్ను ఎం యల్ ఎ కొడుకు నిన్ను వికారాబాద్ అడవిలో చంపేస్తుండా అని ఏవో కూతులు కూస్తా ఉంటే బయపడి పోన్ చెస్తాఉండా,రేయ్ మచ్చా ఏడున్నావ్ రా అంటే ఏమైంది రా అంటే ఎవరు ఎమ్మెల్యే కొడుకు నేను వికారాబాద్ చంపేస్తుంటే చూస్తూ ఉంటే ఫోన్ చేస్తా ఉండు,NaN
1,vad_35_R4kPm3KHUcw,3.03,male,1.0,no,telugu,kadapa,no,general,ఇక్కడ మనలాగా అక్కడపతథియేటర్లో ఎక్కడ ఉంటాయి,ఇక్కడ మునలాగా అక్కడి భర్త కట్ థియేటర్లో ఎక్కడో ఉంటాయి,ఇక్కడ మనలాగ ఎక్కడపడ్తే అక్కడ థియేటర్లు ఉండవు ఎక్కడో ఉంటాయి,ఇక్కడ మన లాగా ఇక్కడికొచ్చాక థియేటర్లో ఎక్కడ ఉంటాయి,NaN
2,broken-vad_05_lshQKEpvm6c-8,13.22,male,2.0,yes,telugu+english,kadapa,yes,general,నుేరా,అమ్మ ఇంతసాయకువగారు,నీయమ్మ ఎంత సైకోగడురా వీడు నువ్వేరా కాదురా ఇటురా దా ముందుకురా ఇదేందిరా రాసినవ్ ఫస్ట్ పేపరులో,ఈ అబ్బాయి ఎంత సేపు పడుతుంది ఫస్ట్ పేపర్ లో,NaN
